In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# set up arrays
dx = 1000
x = np.arange(0, 100000, dx)
x_node = x + (dx/2)

Hstar = 0.5
etab = np.zeros(len(x)) # bedrock elevation, m
etab[:] = np.linspace(10, 0.1, len(x)) # need to start with slight bedrock slope
H = Hstar + np.zeros(len(x))
eta = etab + H

In [3]:
# set up parameters
# K_ig = np.zeros(len(x))
# K_ig[:24] = 1e-7
# K_sed = np.zeros(len(x))
# K_sed[24:] = 1e-6
K = np.zeros(len(x))
K[:] = 1e-7

# beta_ig = np.zeros(len(x))
# beta_ig[:24] = 4e-7
# beta_sed = np.zeros(len(x))
# beta_sed[24:] = 4e-6
beta = np.zeros(len(x))
beta[:] = 1e-5

# psi = np.zeros(len(x))
# psi[:14] = 4e-7
# psi[14:] = 4e-6
psi = 4e-5
gamma = 0.3

r = 10. # runoff rate
D = 0.015 # grain size
phi = 0.45 # porosity
baselevel_rate = 0.0001 # baselevel fall rate

In [4]:
kqs = 0.041
I = 0.01
kxb = 25
Pxb = 1./5.
kb = 8.3e-8

B_valley = kxb * (x_node**Pxb)
B_basin = (2./3.) * x_node

area = (1./3.) * x**2
# Q = r * x * B
Q = r * area

In [5]:
def model(x, dx, x_node,
          K, beta,
          D, phi, Hstar, psi, gamma,
          kb, kqs, I, r, B_valley, B_basin, Q,
          etab, H, eta,
          baselevel_rate,
          num_steps = 1):
    
    q = Q/B_valley
    # dt = 0.2 * dx * dx / (kqs * q[-1]) # try changing to 10, 5, or 1 if this is unstable
    dt = 5
    
    run_duration = dt * num_steps
    cum_time = 0.0
    
    Qs = np.zeros(len(x))
    qs = np.zeros(len(x))
    Eb_p = np.zeros(len(x))
    Eb_a = np.zeros(len(x))
    Eb = np.zeros(len(x))
    Eh = np.zeros(len(x))
    E = np.zeros(len(x))
    ql = np.zeros(len(x))
    
    H[-1] = 0
    E[-1] = baselevel_rate
    q[0] = 0
    Qs[0] = 0
    ql[0] = 0
    
    while cum_time < run_duration:
        
        # drop baselevel
        eta[-1] -= baselevel_rate * dt
        
        # set boundary conditions
        etab[-1] = eta[-1]
        
        # calc slope
        S = -np.diff(eta)/dx
        
        # calc bed exposure
        alpha = np.exp(-H/Hstar)
        
        # calc Rh
#         width = (kb * Q[1:] * (S ** (7/6))) / (D**(3/2))
#         depth = (0.09801 * D) / S
#         Rh_efficiency = 1 - np.exp(-width/(2*depth))
        
        # calc sed transport
        Qs[1:] = kqs * I * Q[1:] * S**(7./6.) * (1 - alpha[:-1]) # * Rh_efficiency
        qs[1:] = Qs[1:] / B_valley[1:]
        
        # calc bedrock erosion
        Eb_p[:-1] = K[1:] * Q[1:] * S * alpha[:-1]
        Eb_a[:-1] = beta[1:] * Qs[1:] * alpha[:-1]
        Eb[:-1] = (Eb_p[:-1] + Eb_a[:-1]) / B_valley[:-1]
        
        # calc grain attrition
        atr = Qs * psi
        
        # calc equilibrium lateral supply
        ql[1:] = ((baselevel_rate * gamma) / psi) * (1 - np.exp(-psi/(3 * x[1:])))
        # ql[24:] = 0
        
        # calc sediment erosion rate
        Eh[:-1] = 1/((1 - phi) * B_valley[:-1]) * (np.diff(Qs)/dx + atr[1:] - Eb_p[:-1] - ql[1:])
        
        # calc total erosion rate
        E[:-1] = Eb[:-1] + Eh[:-1]
        
        # update topography
        etab[:-1] -= Eb[:-1] * dt
        H[:-1] -= Eh[:-1] * dt
        H[H<0] = 0
        eta[:-1] = etab[:-1] + H[:-1]
        
        # update elapsed time
        cum_time += dt
        
    print(cum_time)
        
    return (dt, S, q, alpha, Qs, qs, Eb_p, Eb_a, Eb, atr, ql, Eh, E, etab, H, eta)

In [6]:
#now write a test
num_steps = 5000000
test_run = [x, dx, x_node, 
            K, beta, 
            D, phi, Hstar, psi, gamma,
            kb, kqs, I, r, B_valley, B_basin, Q,
            etab, H, eta,
            baselevel_rate,
            num_steps]

# and run it
(dt, S, q, alpha, Qs, qs, Eb_p, Eb_a, Eb, atr, ql, Eh, E, etab, H, eta) = model(*test_run)

25000000.0


In [7]:
E

array([1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04, 1.e-04,
       1.e-04, 1.e-04, 1.e-04, 1.e-04])

In [8]:
# now check widths
kb = 8.3e-8
b = (kb * Q[1:] * (S ** (7/6))) / (D**(3/2))
print(b)

[ 3.14937175  4.40276947  5.62728314  6.8762219   8.14012893  9.40963494
 10.67775818 11.939367   13.19063957 14.42869133 15.65132582 16.85686451
 18.04402735 19.21184652 20.35960239 21.48677513 22.59300723 23.67807417
 24.74186097 25.78434332 26.80557221 27.80566124 28.78477612 29.74312591
 30.68095561 31.59853995 32.4961781  33.37418913 34.23290816 35.07268305
 35.89387157 36.69683892 37.48195563 38.24959574 39.0001352  39.73395053
 40.45141766 41.15291088 41.83880199 42.50945956 43.16524825 43.80652834
 44.4336552  45.04697891 45.64684397 46.23358901 46.80754656 47.36904291
 47.91839791 48.45592496 48.98193086 49.49671581 50.00057334 50.49379038
 50.97664722 51.44941755 51.91236854 52.36576083 52.80984869 53.24488003
 53.67109648 54.08873353 54.49802061 54.89918116 55.29243276 55.67798725
 56.05605083 56.42682416 56.7905025  57.14727578 57.49732878 57.84084121
 58.17798781 58.50893849 58.83385847 59.15290832 59.46624415 59.77401766
 60.07637632 60.3734634  60.66541813 60.9523758  61

In [10]:
b_list = list(b)
print(b_list)

[3.1493717516889386, 4.40276946593797, 5.627283138260793, 6.876221901220508, 8.140128925450849, 9.409634937986729, 10.67775817799973, 11.93936700197364, 13.190639567485334, 14.428691334797472, 15.651325821958213, 16.856864505352107, 18.044027353874508, 19.211846521896003, 20.359602393629434, 21.486775126840328, 22.593007228497715, 23.678074167461613, 24.741860965937924, 25.784343320450787, 26.805572211770578, 27.805661240699546, 28.78477612226212, 29.743125908210132, 30.68095560833855, 31.598539954197136, 32.49617810438975, 33.37418913073028, 34.23290815789998, 35.07268305168503, 35.893871572437774, 36.69683892309656, 37.48195563415457, 38.24959573904894, 39.00013519716061, 39.73395053359117, 40.45141766410327, 41.152910883337654, 41.838801993319734, 42.50945955615154, 43.165248254229574, 43.80652834491188, 44.433655198732225, 45.04697891025244, 45.64684397312002, 46.23358901221477, 46.807546564526255, 47.36904290536187, 47.91839791221839, 48.455924962772215, 48.98193086397746, 49.4967

In [11]:
# now convert discharge from m^3/yr to m^3/s
Q_pers = Q / 3.15e7

In [13]:
Q_pers

array([0.00000000e+00, 1.05820106e-01, 4.23280423e-01, 9.52380952e-01,
       1.69312169e+00, 2.64550265e+00, 3.80952381e+00, 5.18518519e+00,
       6.77248677e+00, 8.57142857e+00, 1.05820106e+01, 1.28042328e+01,
       1.52380952e+01, 1.78835979e+01, 2.07407407e+01, 2.38095238e+01,
       2.70899471e+01, 3.05820106e+01, 3.42857143e+01, 3.82010582e+01,
       4.23280423e+01, 4.66666667e+01, 5.12169312e+01, 5.59788360e+01,
       6.09523810e+01, 6.61375661e+01, 7.15343915e+01, 7.71428571e+01,
       8.29629630e+01, 8.89947090e+01, 9.52380952e+01, 1.01693122e+02,
       1.08359788e+02, 1.15238095e+02, 1.22328042e+02, 1.29629630e+02,
       1.37142857e+02, 1.44867725e+02, 1.52804233e+02, 1.60952381e+02,
       1.69312169e+02, 1.77883598e+02, 1.86666667e+02, 1.95661376e+02,
       2.04867725e+02, 2.14285714e+02, 2.23915344e+02, 2.33756614e+02,
       2.43809524e+02, 2.54074074e+02, 2.64550265e+02, 2.75238095e+02,
       2.86137566e+02, 2.97248677e+02, 3.08571429e+02, 3.20105820e+02,
      